In [ ]:
# https://pbpython.com/market-basket-analysis.html

In [3]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# DB접속
import os
from os import listdir
import cx_Oracle as oci
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

# 그래프
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
path = 'C:/windows/Fonts/malgun.ttf'                           # 폰트:맑은고딕
font_name = fm.FontProperties(fname=path, size=24).get_name()  
plt.rc('font', family=font_name)

In [5]:
# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_MCRM.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text, encoding='UTF-8', nencoding='UTF-8')

In [7]:
# query run
query1= '''
        with target_sales as (
            select distinct SALE_DT, stor_cd, pos_no, bill_no from SALE_DT
            where SALE_DT = '20220325' and main_item_cd in (180769, 180770, 180771, 180772) )
        select A.saler_dt as InvoiceNo
        , main_item_cd as StockCode
        , (select item_pos_nm from item where item_cd = A.main_item_cd) as Description
        , a.sale_qty as Quantity
        , to_date(A.SALE_DT||A.SALE_TM, 'YYYYMMDDHH24MISS') as InvoiceDate
        from SALE_DT A, target_sales B
        where A.SALE_DT = B.SALE_DT
        and A.stor_cd = B.stor_cd
        and A.pos_no = B.pos_no
        and A.bill_no = B.bill_no
        and A.SALE_DT = '20220325'
        '''
df1_A = pd.read_sql(query1, con = conn )

In [8]:
df1_A

,INVOICENO,STOCKCODE,DESCRIPTION,QUANTITY,INVOICEDATE
0,20220325164733,180772,I 오트 카페라떼 S,1,2022-03-25 16:47:33
1,20220325123015,179978,I소잘라떼S,1,2022-03-25 12:30:10
2,20220325132205,179700,I스페니쉬라떼S,2,2022-03-25 13:21:07
3,20220325142854,170132,I 룽고 G,2,2022-03-25 14:28:25
4,20220325145234,180770,H 오트 카페라떼 S,1,2022-03-25 14:51:53
...,...,...,...,...,...
1463,20220325145425,180772,I 오트 카페라떼 S,1,2022-03-25 14:54:22
1464,20220325170235,271011,우유크림 롤,1,2022-03-25 17:02:35
1465,20220325170235,271011,우유크림 롤,1,2022-03-25 17:02:35
1466,20220325091301,170080,H 카페라떼 S,1,2022-03-25 09:12:46


In [10]:
df1_A['DESCRIPTION'] = df1_A['DESCRIPTION'].str.strip()
df1_A['INVOICENO'] = df1_A['INVOICENO'].astype('str')

In [11]:
basket = (df1_A
          .groupby(['INVOICENO', 'DESCRIPTION'])['QUANTITY']
          .sum().unstack().reset_index().fillna(0)
          .set_index('INVOICENO'))

In [12]:
basket

DESCRIPTION,(P)바스크 치즈케이크,H 룽고 G,H 룽고 S,H 리스트레토,H 스윗밀크티 S,H 시나몬 ECP,H 시나몬라떼 S,H 아메리카노 S,H 오트 카페라떼 G,H 오트 카페라떼 S,...,제주말차 아이스크림(컵),제주말차 혼합 아이스크림(컵),콜드브루 아메리카노300ml,펌킨 식혜 S,폴바셋 스위트 베이글칩,폴바셋단팥빵,플레인마들렌,플로리다 내츄럴오렌지 200ml,햄에그n치즈머핀,흑임자 케이크
INVOICENO,,,,,,,,,,,,,,,,,,,,,
20220325071731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20220325071831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20220325072118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20220325074106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20220325074109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220325203825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20220325204131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20220325205839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
# basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [28]:
frequent_itemsets = apriori(basket_sets, min_support=0.1, use_colnames=True)

In [29]:
frequent_itemsets

,support,itemsets
0,0.267606,(H 오트 카페라떼 S)
1,0.138028,(I 오트 카페라떼 G)
2,0.543662,(I 오트 카페라떼 S)


In [30]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [26]:
rules[ (rules['lift'] >= 0.1) &
       (rules['confidence'] >= 0.1) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
